## Crassification Bot
　　　From SIGNATE　【練習問題】ボットの判別

※ SIGNATEから元データの再配布が認められていないため、このノートブックで解析している元データ(csvファイル、tsvファイル)については、お手数ですがSIGNATEの【練習問題】サイトからダウンロードして、このipynbファイルと同じ場所に入れてから実行してください。

In [ ]:
# 基本ライブラリの読み込み
import numpy as np
import pandas as pd
import seaborn as sns

### trainデータ

In [ ]:
# train.tsvの読み込み
df_train = pd.read_csv("train.tsv", sep="\t")
df_train.head()

In [ ]:
# インデックスとbot（目的変数）を説明変数から削除してtrainXに代入する
trainX = df_train.iloc[:, 2:]
trainX.head()

In [ ]:
trainX.describe()

In [ ]:
trainX.shape

In [ ]:
trainX.info()

In [ ]:
trainX.isnull().sum()

In [ ]:
# 項目間の相関を確認
df_train.corr()

In [ ]:
# 項目間の相関をプロット
sns.pairplot(df_train)

In [ ]:
# botカラムとの相関が±0.25以上のカラムのみ代入
trainX2 = df_train[["default_profile","default_profile_image","geo_enabled","mean_mins_between_tweets"]]
trainX2.head()

In [ ]:
# bot（目的変数）をyに代入する
y = df_train.iloc[:, [1]]
y.head()

### testデータ

In [ ]:
# test.tsvの読み込み
df_test = pd.read_csv("test.tsv", sep="\t")
df_test.head()

In [ ]:
testX = df_test.iloc[:, 1:]
testX.head()

In [ ]:
testX.describe()

In [ ]:
testX.shape

In [ ]:
testX.info()

In [ ]:
testX.isnull().sum()

In [ ]:
# botカラムとの相関が±0.25以上のカラムのみ代入
testX2 = df_test[["default_profile","default_profile_image","geo_enabled","mean_mins_between_tweets"]]
testX2.head()

### 交差検証

In [ ]:
# 交差検証　Holdout法
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(trainX,
                                                 y,
                                                 test_size=0.30,
                                                 random_state=1)

In [ ]:
# 整形
y_train = y_train.values.ravel()
y_test = y_test.values.ravel()

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.decomposition import PCA
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier

# set pipelines for three different algorithms
pipe_LR = Pipeline([('scl',StandardScaler()),
                       ('est',LogisticRegression(random_state=1))])
pipe_RF = Pipeline([('scl',StandardScaler()),
                        ('est',RandomForestClassifier(random_state=1))])
pipe_GB = Pipeline([('scl',StandardScaler()),
                        ('est',GradientBoostingClassifier(random_state=1))])
pipe_PCARF = Pipeline([('scl',StandardScaler()),
                          ('est',PCA(random_state=1)),
                          ('rfc',RandomForestClassifier(random_state=1))])
pipe_SVC = Pipeline([('scl',StandardScaler()),
                          ('est',LinearSVC(random_state=1))])
pipe_KNC = Pipeline([('scl',StandardScaler()),
                          ('est',KNeighborsClassifier())])

In [ ]:
# 学習
pipe_LR.fit(X_train, y_train)
pipe_RF.fit(X_train, y_train)
pipe_GB.fit(X_train, y_train)
pipe_PCARF.fit(X_train, y_train)
pipe_SVC.fit(X_train, y_train)
pipe_KNC.fit(X_train, y_train)

In [22]:
# 評価（f1スコアとする）
from sklearn.metrics import accuracy_score, roc_auc_score, precision_score, recall_score, f1_score

print('pipe_LR_Train:%.3f'% f1_score(y_train,
                                          pipe_LR.predict(X_train)))
print('pipe_LR_Test:%.3f' % f1_score(y_test,
                                          pipe_LR.predict(X_test)))

print('pipe_RF_Train:%.3f'% f1_score(y_train,
                                          pipe_RF.predict(X_train)))
print('pipe_RF_Test:%.3f' % f1_score(y_test,
                                          pipe_RF.predict(X_test)))

print('pipe_GB_Train:%.3f'% f1_score(y_train,
                                          pipe_GB.predict(X_train)))
print('pipe_GB_Test:%.3f' % f1_score(y_test,
                                          pipe_GB.predict(X_test)))

print('pipe_PCARF_Train:%.3f'% f1_score(y_train,
                                          pipe_PCARF.predict(X_train)))
print('pipe_PCARF_Test:%.3f' % f1_score(y_test,
                                          pipe_PCARF.predict(X_test)))

print('pipe_SVC_Train:%.3f'% f1_score(y_train,
                                          pipe_SVC.predict(X_train)))
print('pipe_SVC_Test:%.3f' % f1_score(y_test,
                                          pipe_SVC.predict(X_test)))

print('pipe_KNC_Train:%.3f'% f1_score(y_train,
                                          pipe_KNC.predict(X_train)))
print('pipe_KNC_Test:%.3f' % f1_score(y_test,
                                          pipe_KNC.predict(X_test)))

pipe_LR_Train:0.470
pipe_LR_Test:0.396
pipe_RF_Train:0.964
pipe_RF_Test:0.648
pipe_GB_Train:0.962
pipe_GB_Test:0.720
pipe_PCARF_Train:0.961
pipe_PCARF_Test:0.490
pipe_SVC_Train:0.412
pipe_SVC_Test:0.349
pipe_KNC_Train:0.580
pipe_KNC_Test:0.431


#### Gradient Boosting Classifier を採用する

In [ ]:
# パラメータグリッドの設定
from sklearn.model_selection import GridSearchCV
param_grid_GB = {'est__loss':['deviance','exponential'],
#                  'est__learning_rate':[0.001,0.01,0.1],
#                  'est__n_estimators':[5,10,50,100,500], 
                 'est__max_depth':[1,2,3,4,5],
                 'est__min_samples_split':[0.1,0.3,0.5,0.7,0.9],
                 'est__min_samples_leaf':[1,2,4,6,8] }

# loss：最小化にするための費用関数　標準値deviance
# learning_rate：標準値0.1　結果に対する各ツリーの影響度を小さくする。↓に反比例。小さいほどロバスト性が高く良い
# n_estimators：標準値100　逐次的にフィットするためrのツリー数↑に対して交差検証から調整 
# max_depth：標準値設定なし　各ツリーの最大深さ　
# min_samples_split：標準値2　ノードとして必要な最小サンプル数
# min_samples_leaf：標準値1　リーフとして必要な最小サンプル数

In [24]:
# write your answer
print('探索空間:%s' %param_grid_GB)
clf = GridSearchCV(estimator=pipe_GB,
                   param_grid=param_grid_GB,
                   scoring='f1',
                   cv=3,
                   return_train_score=False)
clf = clf.fit(X_train, y_train)
print('Best Score:', clf.best_score_)
print('Best Params', clf.best_params_)

探索空間:{'est__loss': ['deviance', 'exponential'], 'est__max_depth': [1, 2, 3, 4, 5], 'est__min_samples_split': [0.1, 0.3, 0.5, 0.7, 0.9], 'est__min_samples_leaf': [1, 2, 4, 6, 8]}
Best Score: 0.728403353386644
Best Params {'est__loss': 'exponential', 'est__max_depth': 4, 'est__min_samples_leaf': 8, 'est__min_samples_split': 0.1}


In [ ]:
pred = pipe_GB.predict(testX)
pred

### 提出用CSVの作成

In [ ]:
sample = pd.read_csv("sample_submit.csv",header=None)
sample.head()

In [ ]:
sample[1]=pred

In [ ]:
sample.to_csv("submit3.csv",index=None,header=None)